# Module 2: Atomistic Machine Learning
## Part 2: Equivariance

In this notebook, we illustrate the notion of equivariance of quantities with respect to operations in SO(3)

### 1. Install and load python libraries

In [ ]:
!pip install ase dscribe numpy

In [ ]:
import numpy as np
from ase import Atoms
from ase.build import molecule
from ase.visualize import view
from dscribe.descriptors import CoulombMatrix

### 2. Create molecule
Choose a molecule

In [ ]:
# Create molecule, for instance (CH4, CH3CH2NH2, H2COH ...)
mymolecule = molecule("CH4")
Natoms=len(mymolecule)
view(mymolecule, viewer='x3d')

### 3. Define transformations

In [ ]:
# Function to apply transformations
def rotate_atoms(atoms, angle=90, axis='z'):
    rotated = atoms.copy()
    rotated.rotate(a=angle, v=axis, rotate_cell=False)
    return rotated

def translate_atoms(atoms, vector):
    translated = atoms.copy()
    translated.translate(vector)
    return translated

def permute_atoms(atoms, permutation):
    positions = atoms.get_positions()[permutation]
    symbols = np.array(atoms.get_chemical_symbols())[permutation]
    return Atoms(symbols=symbols, positions=positions)

mypermutation=np.random.permutation(Natoms)

### 4. Define appropriate formula for equivariant quantity
The quantity,
$$
\mathbf{v}_i (\mathbf{R}) = \sum_j f(|\mathbf{r}_{ij}|) \mathbf{r}_{ij} 
$$
is equivariant with respect to rotations. This means that:
$$
\mathcal{R} \left( \mathbf{v}_i(\mathbf{R}) \right ) =  \mathbf{v}_i \left( \mathcal{R} \left( \mathbf{R} \right ) \right ) 
$$
where $\mathcal{R}$ is a rotation.

We will use the Coulomb matrix elements $C_{ij}$ to define the quantity,
$$
\mathbf{v}_i (\mathbf{R}) = \sum_j C_{ij} \mathbf{r}_{ij} 
$$
and test its equivariance.

In [ ]:
# Function to compute vector sum for each atom
def compute_equivariant_vectors(atoms, C):
    positions = atoms.get_positions()
    vectors = np.zeros((len(atoms), 3))
    for i in range(len(atoms)):
        for j in range(len(atoms)):
            if i != j:
                rij = positions[j] - positions[i]
                cij = C[i,j]
                vectors[i] += rij * cij
    return vectors

### 5. Compare vector before and after transformations

In [ ]:
np.set_printoptions(precision=6, suppress=True, floatmode='fixed')

# Compute original vectors
cm_desc = CoulombMatrix(n_atoms_max=Natoms, permutation="none")
C = cm_desc.create(mymolecule).reshape(Natoms,Natoms)
vecs_original = compute_equivariant_vectors(mymolecule, C)

# Rotate atoms and compute vectors again
rotated = rotate_atoms(mymolecule, 90, 'z')
C_rotated = cm_desc.create(rotated).reshape(Natoms,Natoms)
vecs_atoms_rotated = compute_equivariant_vectors(rotated, C_rotated)

# Rotate the original vectors
symbols = ['H'] * Natoms
vecs_original_ase = Atoms(symbols=symbols, positions=vecs_original)
rotated = rotate_atoms(vecs_original_ase, 90, 'z')
vecs_rotated=rotated.get_positions()

print("Original vectors:")
print(vecs_original)
print("Vectors with atoms rotated:")
print(vecs_atoms_rotated)
print("Rotated vectors:")
print(vecs_rotated)

Above, we are comparing the two sides of the equation,
$$
\mathcal{R} \left( \mathbf{v}_i(\mathbf{R}) \right ) =  \mathbf{v}_i \left( \mathcal{R} \left( \mathbf{R} \right ) \right ) 
$$

### 6. Exercise
Test equivariance for tensors